# Morphing

> This module apply morphing to a template source from landmarks. Morphing from pygem library.


In [ ]:
#| default_exp morph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import numpy as np

In [ ]:
#| export
from infepy.rbf import RBF

In [ ]:
#| export
from infepy.preprocessing import read_landmarks, read_nodes, _check_landmarks, write_output
from infepy.utils import read_toml, multiple_targets, _merge_path, to_ls_dyna

In [ ]:
#| export
def morphing(source_mesh: np.ndarray, # Coordinates of the mesh/ mesh 
             source_landmarks: np.ndarray,  # Landmarks of the source mesh
             target_landmarks: np.ndarray,  # Target Landmarks
             ):
    "Morph the target mesh with RBF function - Thin Plate Spine."
    rbf = RBF(original_control_points=source_landmarks, deformed_control_points=target_landmarks,
            func='thin_plate_spline', radius=1.0)
    return rbf(source_mesh)

In [ ]:

# #| export
# def main():
#     config = read_toml()
#     source_landmarks = read_landmarks(_merge_path(config['source']['path'],config['source']['filename_landmarks']))
#     template_mesh = read_nodes(_merge_path(config['source']['path'],config['source']['filename_mesh']))

#     if multiple_targets() == bool: # single target
#         target_landmarks = read_landmarks(_merge_path(config['target']['path'],config['target']['filename_landmarks']))
#         _check_landmarks(source_landmarks, target_landmarks)
#         morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh)
#         write_output(morphed_mesh)
#     else:
#         targets_folder = multiple_targets()
#         for folder in targets_folder:
#             new_path = os.path.join(config['target']['path'],folder, config['target']['filename_landmarks'])
#             target_landmarks = read_landmarks(new_path)
#             _check_landmarks(source_landmarks, target_landmarks)
#             morphed_mesh = morphing(source_landmarks, target_landmarks, template_mesh)
#             write_output(morphed_mesh)
#     return
# main()

NameError: name 'read_csv_file' is not defined

In [ ]:
#| export
# @click.command()
# def hello():
#     click.echo("hello")

In [ ]:
# #| export
# if __name__ == "__main__":
#     main()
    

/mnt/c/Users/chiarar/OneDrive - Chalmers/SSMT - Subject Specific Modeling Toolstack/Github/infepy/nbs
Yes!!
Yes!!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()